In [12]:
import torch
import torch.nn as nn
import torchvision
import torchvision.transforms as transforms
import torch.optim as optim
import torch.nn.functional as F
import numpy as np

from sklearn.metrics import roc_auc_score

In [13]:
# use cuda if available or else use cpu
device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")
print('Active processing device: ', device)

Active processing device:  cpu


In [14]:
# torch.cuda.is_available()
device

device(type='cpu')

In [15]:
# hyper parameters
n_epochs =15
n_classes =10
batch_size = 128
learning_rate = 0.001

In [16]:
# Load MNIST dataset in pytorch
train_dataset = torchvision.datasets.MNIST('../data/', train=True, transform=transforms.ToTensor(), download=True)
test_dataset = torchvision.datasets.MNIST('../data/', train=False, transform=transforms.ToTensor())

In [17]:
# Data loader
train_loader = torch.utils.data.DataLoader(dataset=train_dataset, batch_size=batch_size, shuffle=True)
test_loader = torch.utils.data.DataLoader(dataset=test_dataset, batch_size=batch_size, shuffle=False)

In [18]:
# Create Model
class CNN(nn.Module):
    def __init__(self):
        super(CNN, self).__init__()
        self.conv1 = nn.Conv2d(1, 32, 3, 1)
        self.conv2 = nn.Conv2d(32, 64, 3, 1)
        self.dense1 = nn.Linear(12*12*64, 128)
        self.dense2 = nn.Linear(128, n_classes)

    def forward(self, x):
        x = F.relu(self.conv1(x))
        x = F.relu(self.conv2(x))
        x = F.max_pool2d(x, 2, 2)
        x = F.dropout(x, 0.25)
        x = x.view(-1, 12*12*64) # same as flat layer
        x = F.relu(self.dense1(x))
        x = F.dropout(x, 0.5)
        x = self.dense2(x)
        return F.log_softmax(x, dim=1)



In [9]:
model = CNN().to(device)
ceriterion = nn.CrossEntropyLoss()
optimizer = torch.optim.Adam(model.parameters(), lr=learning_rate)

total_step = len(train_loader)
for epoch in range(n_epochs):
    for i, (images, labels) in enumerate(train_loader):
        images = images.to(device)
        labels = labels.to(device)
        
        # Forward pass
        outputs = model(images)
        loss = ceriterion(outputs, labels)
        
        # Back propagation and optimization
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()
        
        if (i+1) % 100 == 0:
            print(f'Epoch [{epoch + 1}/{n_epochs}], step [{i+1}/{total_step}],\
                    Loss:{loss.item():.4f}')

Epoch [1/15], step [100/469],                    Loss:0.2784
Epoch [1/15], step [200/469],                    Loss:0.3376
Epoch [1/15], step [300/469],                    Loss:0.0731
Epoch [1/15], step [400/469],                    Loss:0.0864
Epoch [2/15], step [100/469],                    Loss:0.1008
Epoch [2/15], step [200/469],                    Loss:0.1180
Epoch [2/15], step [300/469],                    Loss:0.0268
Epoch [2/15], step [400/469],                    Loss:0.1210
Epoch [3/15], step [100/469],                    Loss:0.1316
Epoch [3/15], step [200/469],                    Loss:0.0256
Epoch [3/15], step [300/469],                    Loss:0.0913
Epoch [3/15], step [400/469],                    Loss:0.0565
Epoch [4/15], step [100/469],                    Loss:0.1045
Epoch [4/15], step [200/469],                    Loss:0.0803
Epoch [4/15], step [300/469],                    Loss:0.0999
Epoch [4/15], step [400/469],                    Loss:0.1363
Epoch [5/15], step [100/

In [10]:
# model test
from sklearn.metrics import roc_auc_score

In [11]:
preds = []
y_true = []

# set model to evaluation mode
model.eval()
with torch.no_grad():
    correct = 0
    total = 0
    for images, labels in test_loader:
        images = images.to(device)
        labels = labels.to(device)
        outputs = model(images)
        _ , predicted = torch.max(outputs.data, 1)
        total += labels.size(0)
        correct += (predicted == labels).sum().item()
        detached_pred = predicted.detach().cpu().numpy()
        detached_label = labels.detach().cpu().numpy()
        for f in range(0, len(detached_pred)):
            preds.append(detached_pred[f])
            y_true.append(detached_label[f])
    
    print(f'Test Accuracy on 10k images: {correct/total: .2%}')
    
        
    preds = np.eye(n_classes)[preds]
    y_true = np.eye(n_classes)[y_true]
    auc = roc_auc_score(preds, y_true)
    print(f'AUC: {auc:.2%}')
    
# save model checkpoint
torch.save(model.state_dict(), 'pytorch_mnist_cnn.ckpt')



Test Accuracy on 10k images:  98.46%
AUC: 99.14%
